In [1]:
import findspark
findspark.init('/usr/local/spark/')
import pyspark

Import and initiate findspark to begin with.
And import pyspark

Now start SparkSession

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

Create DataFrame from data source - csv file

In [3]:
customerDF = spark.read.load("customers.txt", format="csv", sep="\t", inferSchema="true", header="true")

Different types of operations on DataFrames

In [4]:
customerDF.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- customer_city: string (nullable = true)
 |-- customer_state: string (nullable = true)
 |-- customer_zipcode: integer (nullable = true)



In [ ]:
customerDF.select("customer_name").show()

In [ ]:
customerDF.select(customerDF['customer_name'], customerDF['customer_city']).show()

In [ ]:
customerDF.filter(customerDF['customer_state'] == 'CA').show()

In [ ]:
customerDF.groupBy("customer_state").count().show()

Create temp view for running SQL queries on the dataframe

In [ ]:
customerDF.createOrReplaceTempView("customers")

SQL can be run on DataFrames that are registered as temp views.

In [ ]:
cStateCount50 = spark.sql("SELECT customer_state, count(*) as state_count FROM customers GROUP BY customer_state HAVING state_count>=50")

In [ ]:
type(cStateCount50)

In [ ]:
cStateCount50.show()

In [ ]:
import pickle

In [ ]:
l = {'ID':[1,2,3],'Name':["a","b","c"],"Sal":[123,90098,183]}

In [ ]:
type(l)

In [ ]:
# Serialization
p = pickle.dumps(l)

In [ ]:
p_deserialized = pickle.loads(p)

In [ ]:
p_deserialized

In [ ]:
cStateCount50.printSchema()

In [ ]:
cStateCount50.write.parquet("cStateunpart.parquet")

In [ ]:
cStateCount50.coalesce(2).write.parquet("cStatepart.parquet")

Since the dataframe is small we can coalesce all the partitions into one and write it. This will result in a single file output else output will be in as many files as the number of partitions. coalesce can prove expensive on large dataframes.

Alternatively the following syntax is also allowed.
We can use the above since parquet is the default format used by SparkSQL.

cStateCount50.coalesce(1).write.save("cStateOutput2.parquet", format="parquet")

cStateCount50.coalesce(1).write.save("cStateOutput3.parquet")

In [ ]:
cStateCount50.coalesce(1).write.json("cState.json")

Alternatively the following syntax is also allowed.

cStateCount50.coalesce(1).write.save("cStateOutput2.json", format="json")

Now Create DataFrame from data source - JSON file

In [ ]:
productDF = spark.read.load("products.json", format="json")

Alternatively the following syntax is also allowed

productDF = spark.read.json("products.json")

We can now run set of the DataFrame operations

In [ ]:
productDF.printSchema()

In [ ]:
productDF.select("product_name").show()

In [ ]:
productDF.select(productDF['product_name'], productDF['product_category'], productDF['product_price']).show()

In [ ]:
productDF.filter(productDF['product_price'] > 200.00).show()

In [ ]:
productDF.groupBy("product_category").count().show()

Create temp view for running SQL queries on the dataframe

In [ ]:
productDF.createOrReplaceTempView("products")

SQL queries can be run on the DataFrames that have been registered as a table.

In [ ]:
prd200 = spark.sql("SELECT category_id, product_category, count(*) as prdcount FROM products WHERE product_price>200 GROUP BY category_id, product_category ORDER BY product_category")

In [ ]:
type(prd200)

In [ ]:
prd200.show()

In [ ]:
prd200.printSchema()

In [ ]:
prd200.coalesce(1).write.save("product3.parquet")

Alternatively the following syntax is also allowed

prd200.write.save("product2.parquet", format="parquet")

prd200.write.parquet("product1.parquet")

Since parquet is the default format used by SparkSQL we need not specify it as shown in the first command.

In [ ]:
prd200.coalesce(1).write.json("product1.json")

Alternatively the following syntax is also allowed

prd200.write.save("product2.json", format="json")

Now that we have two datasets in two views we can join them on the common column for queries. For example:
    
Get the list of customers and product categories in which they bought multiple items (quantity) that are more expensive than 200.00

In [ ]:
custlist200 = spark.sql("SELECT a.customer_name, b.product_category, count(*) as prdcount FROM customers a INNER JOIN products b ON a.customer_id=b.customer_id WHERE b.product_price>200.00 GROUP BY a.customer_name, b.product_category HAVING prdcount>1")

In [ ]:
type(custlist200)

In [ ]:
custlist200.show()

In [ ]:
custlist200.printSchema()

In [ ]:
df = spark.read.parquet("cStatepart.parquet/")

In [ ]:
type(df)

In [ ]:
df.show()

Syntax for reading a parquet file and loading as a DF

productDF1 = spark.read.load("products.parquet")

We can use the above since parquet is the default format used by SparkSQL

productDF2 = spark.read.load("products.parquet", format="parquet")

productDF3 = spark.read.parquet("products.parquet")